Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler() #em geral, redes neurais gostam de dados normalizados 
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32) #array de numpy que calcula o grafo e o gradiente local
y_treino = torch.tensor(y_treino, dtype=torch.float32) #sempre float32, float64 dá erro
X_teste = torch.tensor(X_teste, dtype=torch.float32) #a precisão de float32, na maioria dos casos, é suficiente
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module): #seu modelo é uma classe. classes. existem classes mãe.
    #a classe MLP herda as características de "nn.Module"
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__() #inicializar o init de nn.Module, imutável

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1), #camada
                    nn.ReLU(), #função de ativação, sempre entre duas camadas. 'Rectifier Linear Unit' == ReLU
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )
        #A ReLU é PARCIALMENTE LINEAR. ativar e desativar neurônios. MUITO melhor que a sigmoidal.
        #"Usa a ReLU, se não der pensa em outra" -- Daneil R. Cassar

    def forward(self, x): #forward pass
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1] #quantas colunas
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 0.1833,  0.2476,  0.1210,  0.1204,  0.3195, -0.2441],
        [-0.3550, -0.2226, -0.1683, -0.1202, -0.3518,  0.2683],
        [-0.2845,  0.1907, -0.2987,  0.1682, -0.1336,  0.0133],
        [-0.1082, -0.2889, -0.2156, -0.0317, -0.1031, -0.2716],
        [ 0.3934, -0.2890,  0.2081,  0.2540, -0.0762,  0.0226],
        [-0.2213, -0.0206,  0.0952,  0.1617, -0.2682, -0.1072],
        [ 0.2267, -0.3632, -0.3408, -0.0280,  0.1836,  0.3204],
        [ 0.1174,  0.0690, -0.3676, -0.3290, -0.0549, -0.2477],
        [-0.2232,  0.0436, -0.2011, -0.3611,  0.0612, -0.3299],
        [-0.0136,  0.0328, -0.2268, -0.2791, -0.1114,  0.1196],
        [-0.0062, -0.3867,  0.0326, -0.3080, -0.0451,  0.2536],
        [ 0.2176,  0.0919,  0.3443, -0.0160, -0.2625, -0.3243],
        [-0.3406, -0.1672, -0.3944,  0.1918,  0.3175,  0.0182],
        [ 0.2062, -0.2388, -0.1791, -0.2000, -0.0239,  0.2464],
        [-0.2546,  0.0664, -0.3978, -0.0502, -0.0408, -0.3261],
        [-0.3736, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.0333],
        [0.0340],
        [0.0317],
        ...,
        [0.0379],
        [0.0258],
        [0.0285]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente #ADAMSKA
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO) #o otimizador cuida dos parâmetros

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train() #'modo de treino' do pytorch, não significa estar treinando, isso não é treinar

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0693)
1 tensor(0.0661)
2 tensor(0.0630)
3 tensor(0.0600)
4 tensor(0.0573)
5 tensor(0.0547)
6 tensor(0.0522)
7 tensor(0.0500)
8 tensor(0.0479)
9 tensor(0.0461)
10 tensor(0.0444)
11 tensor(0.0429)
12 tensor(0.0416)
13 tensor(0.0405)
14 tensor(0.0395)
15 tensor(0.0387)
16 tensor(0.0380)
17 tensor(0.0374)
18 tensor(0.0369)
19 tensor(0.0364)
20 tensor(0.0361)
21 tensor(0.0357)
22 tensor(0.0354)
23 tensor(0.0350)
24 tensor(0.0347)
25 tensor(0.0344)
26 tensor(0.0340)
27 tensor(0.0335)
28 tensor(0.0331)
29 tensor(0.0325)
30 tensor(0.0320)
31 tensor(0.0314)
32 tensor(0.0309)
33 tensor(0.0303)
34 tensor(0.0298)
35 tensor(0.0293)
36 tensor(0.0289)
37 tensor(0.0285)
38 tensor(0.0280)
39 tensor(0.0276)
40 tensor(0.0272)
41 tensor(0.0267)
42 tensor(0.0263)
43 tensor(0.0258)
44 tensor(0.0253)
45 tensor(0.0248)
46 tensor(0.0242)
47 tensor(0.0237)
48 tensor(0.0232)
49 tensor(0.0227)
50 tensor(0.0221)
51 tensor(0.0216)
52 tensor(0.0211)
53 tensor(0.0206)
54 tensor(0.0201)
55 tensor(0.0195)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad(): #no teste, calcular o gradiente não é necessário. usar a rede.
    y_true = normalizador_y.inverse_transform(y_treino) #'desnormalizar'
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1471.70721674]
[1409.99999118] [1523.45513821]
[1128.99997044] [1184.52956247]
[12991.99967575] [11587.37130356]
[14720.0001564] [14402.26045227]
[2938.99994564] [2679.86552429]
[6635.00014687] [7997.33918381]
[11215.99973297] [9610.34336472]
[1040.99999189] [1443.9930644]
[5040.00003624] [5128.07677269]
[9541.99988747] [8744.79859543]
[3792.99988461] [5446.07003975]
[1002.0000155] [767.81539381]
[2542.00005341] [1630.45965815]
[6788.00001717] [6760.86106682]
[9386.00025749] [8794.54543686]
[1245.99996853] [1771.31069469]
[997.00000262] [521.94613767]
[708.00000262] [756.99004829]
[1435.99997544] [1457.7069602]
[2590.00006676] [2673.34730339]
[1392.99997497] [1700.70223904]
[11755.99946976] [9589.73531342]
[2007.99997377] [1370.65632677]
[2707.99998474] [2616.41567373]
[631.00000417] [661.20172596]
[652.99999881] [935.00972366]
[2492.00006247] [2595.18460178]
[594.99999416] [573.43115985]
[760.0000056] [1071.60338354]
[1179.00003028] [1049.24988222]
[1789.9999361] [181

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval() #modo avaliação
#nesse modo, vários cálculos não necessários não são necessários

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [837.4838407]
[628.00000334] [704.85062015]
[442.00000328] [739.10266519]
[2284.99997044] [2562.60024977]
[1040.99999189] [838.31436265]
[809.99999654] [1234.19537401]
[3049.99987316] [3375.38477516]
[720.00000596] [658.28492546]
[877.99999249] [689.42830968]
[1292.00000429] [1647.63951111]
[7056.00004578] [5113.2747612]
[4176.99999142] [5820.877388]
[13701.00020599] [14867.82844543]
[2948.0000515] [4048.39811134]
[16732.99985504] [13984.04588699]
[1815.00000048] [2048.45387173]
[16220.00016022] [11860.59638214]
[5967.00019073] [4673.6115284]
[473.00000042] [663.6413734]
[1078.9999795] [998.21166134]
[452.00000319] [634.9663347]
[801.99998283] [784.30060148]
[2458.99994993] [2514.89128256]
[2492.99998236] [2569.29458666]
[582.00000203] [709.38288522]
[1405.9999671] [1465.67600203]
[8715.99991035] [11384.44709778]
[7634.00004578] [6579.06581879]
[506.99999839] [650.765131]
[665.99999094] [761.22437692]
[1815.00000048] [1643.03817081]
[707.00001383] [617.6336894]
[599.0000

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1394.5827770645856


Anotações:$ \\ $
REDES NEURAIS SEMPRE SÃO PROPENSAS A OVERFITTING POIS O LOSS SEMPRE DIMINUI (pode haver alguma exceção) !!!!!
Número de épocas: conjunto de validação -> 10% do treino. Não são valores do teste e nem treino. Cálculo do "loss de validação",  ele nem sempre diminui. para-se no mínimo. os dados de validação são os "termômetros", não se consegue observar o overfitting.
Nota: no overfitting, a rede não prevê, ela decora os dados.
Processo de Batching: "loteamento" dos dados por meio de passar os dados por meio de um passo. Para cada lote tem uma perda associada. A ordem dos lotes importa, pode-se escolhê-los de de forma aleatória.

## Conclusão



Este notebook tem o objetivo de treinar uma rede neural por meio do Pytorch. Nesse sentido, utilizou-se tensores como substituto para a Classe "Valores", a Classe MLP (que herda características de nn.Module) como substituto para as Classes Neuronio, Camada e MLP. Além disso, foi possível importar a função de perda (nesse caso o MSE) ao invés de programá-la manualmente. Outro ponto interessante é a possibilidade da utilização do Adam, um tipo de algoritmo para descida de gradiente. Além do mais (agora) é possível facilmente alterar a função de ativação, uma vez que anteriormente esta estava atrelada à Classe "Neuronio". Com o Pytorch, pode-se facilmente atribuir uma função de ativação para camada, de modo que neste notebook foi utilizado a ReLU (pra mim parece nome de vaca, mas deixa isso em off). É preciso frisar que, em teoria, pode-se estabelecer uma função de ativação por neurônio, mas isso seria trabalhoso e provavelmente, se extremamente bem realizado, o pináculo de otimização. Assim, percebeu-se que o pytorch tem uma abordagem, em geral, mais enxuta e mais eficiente, com diversos recursos e importações possíveis para o treinamento de redes neurais. Além do mais, conseguiu-se treinar uma rede neural, por meio do pytoch, com base no dataset "diamonds" do Seaborn (um clássico, de fato). Por último, foi fundamental a programação por python puro em um primeiro momento, uma vez que agora, mesmo que o pytorch seja mais enxuto, tem-se uma percepção mais clara do que ocorre nas _hidden layers_ de uma _MLP_.

## Playground

